In [1]:
import os
import time
import torch
import gym
import numpy as np
from gym import wrappers
from PIL import Image

from TD3.td3 import TD3
from TD3.utils import PrioritizedReplayBuffer, mkdir

In [2]:
env_name = 'BipedalWalker-v2'
lr_base = 0.0001
lr_decay = 0.0001
exp_noise_base = 0.1
exp_noise_decay = 0.0001

random_seed = 42
gamma = 0.99                # discount for future rewards
batch_size = 128        # num of transitions sampled from replay buffer
polyak = 0.995              # target policy update parameter (1-tau)
policy_noise = 0.2          # target policy smoothing noise
noise_clip = 0.5
policy_delay = 2            # delayed policy updates parameter
max_episodes = 10000         # max num of episodes
max_timesteps = 2000        # max timesteps in one episode
max_buffer_length = 5000000
log_interval = 10           # print avg reward after interval

In [3]:
actor_config = [
        {'dim': [None, 64], 'dropout': False, 'activation': 'relu'},
        {'dim': [64, 64], 'dropout': False, 'activation':'relu'},
        {'dim': [64, None], 'dropout': False, 'activation': 'tanh'}
    ]
    
critic_config = [
        {'dim': [None, 64], 'dropout': False, 'activation': 'relu'},
        {'dim': [64, 64], 'dropout': False , 'activation':'relu'},
        {'dim': [64, 1], 'dropout': False, 'activation': False}
    ]

In [4]:
class TD3Trainer():
    
    def __init__(self, env_name, actor_config, critic_config, random_seed=42, lr_base=0.001, lr_decay=0.00005, 
                 exp_noise_base=0.3, exp_noise_decay=0.0001, gamma=0.99, batch_size=1024, 
                 polyak=0.9999, policy_noise=0.2, noise_clip=0.5, policy_delay=2, 
                 max_episodes=100000, max_timesteps=3000, max_buffer_length=5000000, 
                 log_interval=5, threshold=None, lr_minimum=1e-10, exp_noise_minimum=1e-10,
                 record_videos=True, record_interval=100,  beta_multiplier=0.0001):        
        
        self.algorithm_name = 'td3'
        self.env_name = env_name
        self.env = gym.make(env_name)
        self.record_videos = record_videos
        self.record_interval = record_interval        
        if self.record_videos == True:
            videos_dir = mkdir('.', 'videos')
            monitor_dir = mkdir(videos_dir, self.algorithm_name)
            should_record = lambda i: self.should_record
            self.env = wrappers.Monitor(self.env, monitor_dir, video_callable=should_record, force=True)            
        self.state_dim = self.env.observation_space.shape[0]
        self.action_dim = self.env.action_space.shape[0]
        self.action_low = self.env.action_space.low
        self.action_high = self.env.action_space.high        
        self.should_record = False
        if not threshold == None:
            self.threshold = threshold
        else:    
            self.threshold = self.env.spec.reward_threshold
        
        self.actor_config = actor_config
        self.critic_config = critic_config
        self.actor_config[0]['dim'][0] = self.state_dim
        self.actor_config[-1]['dim'][1] = self.action_dim
        self.critic_config[0]['dim'][0] = self.state_dim + self.action_dim
        
        self.actor_config = actor_config
        self.critic_config = critic_config
        self.random_seed = random_seed
        self.lr_base = lr_base
        self.lr_decay = lr_decay   
        self.lr_minimum = lr_minimum
        self.exp_noise_base = exp_noise_base
        self.exp_noise_decay = exp_noise_decay     
        self.exp_noise_minimum = exp_noise_minimum                
        self.gamma = gamma
        self.batch_size = batch_size        
        self.polyak = polyak
        self.beta_multiplier = beta_multiplier
        self.policy_noise = policy_noise
        self.noise_clip = noise_clip
        self.policy_delay = policy_delay
        self.max_episodes = max_episodes
        self.max_timesteps = max_timesteps
        self.max_buffer_length = max_buffer_length
        self.log_interval = log_interval
        
        prdir = mkdir('.', 'preTrained')
        self.directory = mkdir(prdir, self.algorithm_name)
        self.filename = "{}_{}_{}".format(self.algorithm_name, self.env_name, self.random_seed)
                
        self.policy = TD3(self.actor_config, self.critic_config, self.action_low, self.action_high)   
        self.replay_buffer = PrioritizedReplayBuffer(size=self.max_buffer_length, alpha=0.8)
        
        self.reward_history = []
        self.make_plots = False       
        
        if self.random_seed:
            print("Random Seed: {}".format(self.random_seed))
            self.env.seed(self.random_seed)
            torch.manual_seed(self.random_seed)
            np.random.seed(self.random_seed)
        
    def train(self):
        
        start_time = time.time()
        print("Training started ... \n")
        print("action_space={}".format(self.env.action_space))
        print("obs_space={}".format(self.env.observation_space))
        print("threshold={}".format(self.threshold))     
        print("action_low={} action_high={} \n".format(self.action_low, self.action_high))         

        # loading models
        self.policy.load(self.directory, self.filename)
                
        # logging variables:        
        log_f = open("train_{}.txt".format(self.algorithm_name), "w+")

        # training procedure:
        for episode in range(1, self.max_episodes+1):
            
            # Only record video during evaluation, every n steps
            if episode % self.record_interval == 0:
                self.should_record = True
            
            ep_reward = 0.0
            state = self.env.reset()
            
            # calculate params
            exploration_noise = max(self.exp_noise_base / (1.0 + episode * self.exp_noise_decay), self.exp_noise_minimum)
            learning_rate = max(self.lr_base / (1.0 + episode * self.lr_decay), self.lr_minimum)     
            beta = min(episode * self.beta_multiplier, 1)
            self.policy.set_optimizers(lr=learning_rate)

            for t in range(self.max_timesteps):
                
                # select action and add exploration noise:
                action = self.policy.select_action(state)               
                action = action + np.random.normal(0, exploration_noise, size=self.action_dim)
                action = action.clip(self.action_low, self.action_high)

                # take action in env:
                next_state, reward, done, _ = self.env.step(action)
                self.replay_buffer.add(state, action, reward, next_state, float(done))
                state = next_state

                ep_reward += reward

                # if episode is done then update policy:
                if done or t==(self.max_timesteps-1):
                    self.policy.update(self.replay_buffer, t, self.batch_size, self.gamma, self.polyak, 
                                       self.policy_noise, self.noise_clip, self.policy_delay, beta)
                    break

            self.reward_history.append(ep_reward)
            avg_reward = np.mean(self.reward_history[-100:]) 

            # logging updates:        
            log_f.write('{},{}\n'.format(episode, ep_reward))
            log_f.flush()
            
            # Calculate polyak
            #part = (env.spec.reward_threshold - avg_reward) / (env.spec.reward_threshold + 150)
            #if part > 1:
            #    part = 1
            #polyak = polyak_int[0] + (1 - part) * (polyak_int[1] - polyak_int[0])     

            # Calculate LR
            #part = min((env.spec.reward_threshold - avg_reward) / (env.spec.reward_threshold + 150), 1)
                        
            avg_actor_loss = np.mean(self.policy.actor_loss_list[-100:])
            avg_Q1_loss = np.mean(self.policy.Q1_loss_list[-100:])
            avg_Q2_loss = np.mean(self.policy.Q2_loss_list[-100:])

            # Truncate training history if we don't plan to plot it later
            if not self.make_plots:
                self.policy.truncate_loss_lists() 
                if len(self.reward_history) > 100:
                    self.reward_history.pop(0)    

            # Print avg reward every log interval:
            if episode % self.log_interval == 0:            
                self.policy.save(self.directory, self.filename)
                print("Ep:{}   Rew:{:3.2f}  Avg Rew:{:3.2f}  LR:{:8.8f}   Polyak:{:5.5f}  Bf:{:2.0f} {:0.4f}  EN:{:0.4f}  Loss: {:5.3f} {:5.3f} {:5.3f}".format(
                    episode, ep_reward, avg_reward, learning_rate, self.polyak, self.replay_buffer.get_fill(), beta, 
                    exploration_noise, avg_actor_loss, avg_Q1_loss, avg_Q2_loss))
                
            self.should_record = False    
                
            # if avg reward > threshold then save and stop traning:
            if avg_reward >= self.threshold and episode > 100: 
                print("Ep:{}   Rew:{:3.2f}  Avg Rew:{:3.2f}  LR:{:8.8f}   Polyak:{:5.5f}  Bf:{:2.0f} {:0.4f}  EN:{:0.4f}  Loss: {:5.3f} {:5.3f} {:5.3f}".format(
                    episode, ep_reward, avg_reward, learning_rate, self.polyak, self.replay_buffer.get_fill(), beta,
                    exploration_noise, avg_actor_loss, avg_Q1_loss, avg_Q2_loss))
                print("########## Solved! ###########")
                name = self.filename + '_solved'
                self.policy.save(self.directory, name)
                log_f.close()
                training_time = time.time() - start_time
                print("Training time: {:6.2f} sec".format(training_time))
                break    
       
    def test(self, episodes=3, render=True, save_gif=True):   
        
        gifdir = mkdir('.','gif')
        algdir = mkdir(gifdir, self.algorithm_name)

        for episode in range(1, episodes+1):
            ep_reward = 0.0
            state = self.env.reset()
            epdir = mkdir(algdir, str(episode))
            
            for t in range(self.max_timesteps):
                action = self.policy.select_action(state)
                state, reward, done, _ = self.env.step(action)
                ep_reward += reward
                
                if save_gif:                                       
                    img = self.env.render(mode = 'rgb_array')
                    img = Image.fromarray(img)
                    img.save('{}/{}.jpg'.format(epdir, t))
                if done:
                    break
                    

            print('Test episode: {}\tReward: {:4.2f}'.format(episode, ep_reward))           
            self.env.close()        
            

In [5]:
agent = TD3Trainer(env_name, actor_config, critic_config, random_seed=random_seed, lr_base=lr_base, lr_decay=lr_decay, 
                   exp_noise_base=exp_noise_base, exp_noise_decay=exp_noise_decay, gamma=gamma, batch_size=batch_size,
                   polyak=polyak, policy_noise=policy_noise, noise_clip=noise_clip, policy_delay=policy_delay, 
                   max_episodes=max_episodes, max_timesteps=max_timesteps, max_buffer_length=max_buffer_length, 
                   log_interval=log_interval)
agent.train()

ACTOR=Sequential(
  (0): Linear(in_features=24, out_features=64, bias=True)
  (1): ReLU()
  (2): Linear(in_features=64, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=4, bias=True)
  (5): Tanh()
)
ACTOR=Sequential(
  (0): Linear(in_features=24, out_features=64, bias=True)
  (1): ReLU()
  (2): Linear(in_features=64, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=4, bias=True)
  (5): Tanh()
)
CRITIC=Sequential(
  (0): Linear(in_features=28, out_features=64, bias=True)
  (1): ReLU()
  (2): Linear(in_features=64, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=1, bias=True)
)
CRITIC=Sequential(
  (0): Linear(in_features=28, out_features=64, bias=True)
  (1): ReLU()
  (2): Linear(in_features=64, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=1, bias=True)
)
CRITIC=Sequential(
  (0): Linear(in_features=28, out_features=64, bias=True)
  (1): Re

Ep:560   Rew:-104.25  Avg Rew:-115.37  LR:0.00009470   Polyak:0.99500  Bf:10 0.0560  EN:0.0947  Loss: 11.857 0.611 0.578
Ep:570   Rew:-150.69  Avg Rew:-119.69  LR:0.00009461   Polyak:0.99500  Bf:11 0.0570  EN:0.0946  Loss: 11.776 0.538 0.507
Ep:580   Rew:-120.96  Avg Rew:-121.81  LR:0.00009452   Polyak:0.99500  Bf:11 0.0580  EN:0.0945  Loss: 11.707 0.381 0.343
Ep:590   Rew:-72.95  Avg Rew:-121.44  LR:0.00009443   Polyak:0.99500  Bf:11 0.0590  EN:0.0944  Loss: 12.073 0.409 0.393
Ep:600   Rew:-98.51  Avg Rew:-122.68  LR:0.00009434   Polyak:0.99500  Bf:12 0.0600  EN:0.0943  Loss: 11.667 0.438 0.407
Ep:610   Rew:-115.65  Avg Rew:-121.88  LR:0.00009425   Polyak:0.99500  Bf:12 0.0610  EN:0.0943  Loss: 10.893 0.401 0.332
Ep:620   Rew:-130.82  Avg Rew:-121.76  LR:0.00009416   Polyak:0.99500  Bf:12 0.0620  EN:0.0942  Loss: 10.171 0.386 0.339
Ep:630   Rew:-129.93  Avg Rew:-123.07  LR:0.00009407   Polyak:0.99500  Bf:13 0.0630  EN:0.0941  Loss: 9.496 0.306 0.314
Ep:640   Rew:-86.43  Avg Rew:-123.6

Ep:1250   Rew:-119.60  Avg Rew:-127.12  LR:0.00008889   Polyak:0.99500  Bf:29 0.1250  EN:0.0889  Loss: 1.824 0.445 0.426
Ep:1260   Rew:-74.75  Avg Rew:-125.24  LR:0.00008881   Polyak:0.99500  Bf:29 0.1260  EN:0.0888  Loss: 3.200 0.252 0.263
Ep:1270   Rew:-56.19  Avg Rew:-120.86  LR:0.00008873   Polyak:0.99500  Bf:29 0.1270  EN:0.0887  Loss: 4.309 0.547 0.605
Ep:1280   Rew:-48.97  Avg Rew:-116.25  LR:0.00008865   Polyak:0.99500  Bf:30 0.1280  EN:0.0887  Loss: 5.053 0.263 0.262
Ep:1290   Rew:-77.66  Avg Rew:-112.32  LR:0.00008857   Polyak:0.99500  Bf:30 0.1290  EN:0.0886  Loss: 5.698 0.405 0.408
Ep:1300   Rew:-38.91  Avg Rew:-103.18  LR:0.00008850   Polyak:0.99500  Bf:30 0.1300  EN:0.0885  Loss: 6.415 0.333 0.305
Ep:1310   Rew:-64.68  Avg Rew:-92.02  LR:0.00008842   Polyak:0.99500  Bf:31 0.1310  EN:0.0884  Loss: 6.675 0.292 0.325
Ep:1320   Rew:-38.52  Avg Rew:-83.00  LR:0.00008834   Polyak:0.99500  Bf:31 0.1320  EN:0.0883  Loss: 6.887 0.246 0.244
Ep:1330   Rew:-47.20  Avg Rew:-70.72  LR:

OSError: [Errno 12] Cannot allocate memory

In [ ]:
agent.test()